# Variational Quantum Eigensolver (VQE) for H₂ Molecule

This notebook demonstrates VQE to find the ground state energy of the Hydrogen molecule (H₂) using a simple qubit Hamiltonian and classical optimization. Uses NumPy for simulation (no real quantum computer needed).

**Key Concepts:**
- Hamiltonian for H₂ at 0.7 Å bond length (minimal basis).
- UCCSD ansatz (simplified to single parameter).
- Classical optimizer (COBYLA) to minimize <ψ| H |ψ>.

Install if needed: `pip install numpy scipy` (but it's standard).

In [ ]:
import numpy as np
from scipy.optimize import minimize

# Simplified H2 Hamiltonian in Pauli basis (2 qubits, minimal STO-3G at r=0.7 Å)
# Terms: constants + Pauli strings (I,I), (X,X), (Y,Y), (Z,Z), etc.
h2_hamiltonian = {
    'II': 0.42,   # Identity
    'ZZ': 0.715,  # Z on qubit 0, Z on 1
    'YY': -0.18,  # Y0 Y1
    'XX': -0.18,  # X0 X1
    'ZI': -1.25,  # Z0 I1
    'IZ': -1.25   # I0 Z1
}

# Pauli matrices
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
I = np.eye(2)

# Expectation value <ψ| H |ψ>
def expectation(params, hamiltonian):
    theta = params[0]
    # Simple ansatz: |ψ> = cos(θ/2)|00> + sin(θ/2)|11> (Hartree-Fock like for H2)
    c0 = np.cos(theta / 2)
    c1 = np.sin(theta / 2)
    psi = np.kron([c0, 0], [c1, 0])  # Wait, proper: |00> and |11>
    psi = np.array([c0, 0, 0, c1])  # Basis: 00,01,10,11
    
    # Compute <H>
    exp_val = 0.0
    for pauli, coeff in hamiltonian.items():
        op = I
        for p in pauli:
            if p == 'X': op = np.kron(op, X)
            elif p == 'Y': op = np.kron(op, Y)
            elif p == 'Z': op = np.kron(op, Z)
            else: op = np.kron(op, I)
        exp_val += coeff * (np.conj(psi) @ op @ psi).real
    return exp_val.real

# Optimize
def run_vqe():
    initial_params = [0.0]
    result = minimize(expectation, initial_params, args=(h2_hamiltonian,), method='COBYLA', options={'maxiter': 100})
    return result.fun, result.x

# Run VQE
ground_energy, optimal_params = run_vqe()
print(f"Ground state energy: {ground_energy:.4f} Hartree")
print(f"Optimal parameter: {optimal_params[0]:.4f}")
print("(Literature value ~ -1.136 Hartree for H2 at 0.7 Å)")

Execute the cell to run VQE! This converges to the ground energy. For full quantum, use Qiskit/Pennylane on real hardware.